<a href="https://www.kaggle.com/code/gabrielfcarvalho/ocr-plate-api?scriptVersionId=254433259" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

# PaddleOCR License Plate Detection and Vehicle Information Retrieval on Brazilian Vehicles

# 📌 **Notebook Overview**
# This notebook demonstrates how to:
# 1️⃣ **Install and import the required libraries.**
# 2️⃣ **Initialize the PaddleOCR model and define helper functions for license plate detection.**
# 3️⃣ **Test the OCR model on a sample image and display detected license plates.**
# 4️⃣ **Retrieve vehicle information from an external API using the detected license plate.**
# 5️⃣ **Display and analyze the retrieved vehicle information.**

# 📌 1️⃣ **Install and import the required libraries.**

In [ ]:
!pip install paddlepaddle==2.6.1
!pip install requests
!pip install opencv-python

In [ ]:
!pip install paddleocr==2.7.3

In [ ]:
import requests
from paddleocr import PaddleOCR
from PIL import Image
import matplotlib.pyplot as plt
import os
import re
import xml.etree.ElementTree as ET

# 📌 2️⃣ **Initialize the PaddleOCR model and define helper functions for license plate detection.**

In [ ]:
# Initialize OCR model
ocr = PaddleOCR(lang='en')

In [ ]:
def is_valid_plate(text):
    # Brazilian old format: "ABC-1234"
    old_format = re.fullmatch(r"[A-Z]{3}-?\d{4}", text)

    # Brazilian new format: "ABC1D23" where second numeric digit (from old format) is converted to a letter
    new_format = re.fullmatch(r"[A-Z]{3}\d[A-Z]\d{2}", text)

    return old_format or new_format

In [ ]:
## 2. Load OCR Model for License Plate Detection
def extract_license_plate(image_path):
    results = ocr.ocr(image_path)
    plates = []
    for res in results:
        for line in res:
            text, confidence = line[1][0], line[1][1]
            text = text.replace(" ", "").upper()  # Normalize text
            if is_valid_plate(text):
                plates.append(text.replace("-", ""))  # Remove hyphen for API compatibility
    return plates

# 📌 3️⃣ **Test the OCR model on a sample image and display detected license plates.**

In [ ]:
## 3. Test OCR Model on a Sample Images
folder_path = "/kaggle/input/brazilian-plates-for-ocr/Placas"
image_path = os.path.join(folder_path, "placa_professor.jpg")

if os.path.exists(image_path):
    img = Image.open(image_path)
    license_plates = extract_license_plate(image_path)

    # Display the image and OCR result
    plt.figure(figsize=(10, 5))
    plt.imshow(img)
    plt.axis("off")
    plt.title(f"Detected License Plates: {license_plates}")
    plt.show()

    print(f"Image: {image_path}")
    print("Detected License Plates:", license_plates)
else:
    print("Image placa_professor.jpg not found.")

# 📌 4️⃣ **Retrieve vehicle information from an external API using the detected license plate.**

In [ ]:
def get_vehicle_info(plate, username):
    api_url = f"https://www.regcheck.org.uk/api/reg.asmx/CheckBrazil?RegistrationNumber={plate}&username={username}"
    response = requests.get(api_url)

    if response.status_code == 200:
        try:
            root = ET.fromstring(response.text)
            vehicle_data = {}

            for elem in root.iter():
                if elem.tag.endswith("vehicleData"):
                    for data in elem:
                        vehicle_data[data.tag.split("}")[-1]] = data.text

            return vehicle_data
        except ET.ParseError:
            print("Error parsing XML response.")
            return None
    else:
        print("Error fetching vehicle data:", response.status_code, response.text)
        return None

In [ ]:
if license_plates:
    username = "username"  # Replace with your registered username
    vehicle_data = get_vehicle_info(license_plates[0], username)
    print("Vehicle Data:", vehicle_data)
else:
    print("No valid license plate detected.")

# 📌 5️⃣ **Display and analyze the retrieved vehicle information.**

In [ ]:
## 4. Display and Analyze Retrieved Vehicle Information
if vehicle_data:
    print("\n--- Vehicle Information ---")
    print(f"Model: {vehicle_data.get('CarModel', 'N/A')}")
    print(f"Year: {vehicle_data.get('RegistrationYear', 'N/A')}")